# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 1.0


In [6]:
brain_name

'BananaBrain'

In [7]:
from dqn_agent3 import Agent
from collections import deque
BATCH_SIZE = 64
BETA = 0
import torch

agent = Agent(state_size=37, action_size=4, seed=0)

def dqn(checkpoint_file, scores_file, n_episodes=1000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, BETA = 0.5):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing sвcores from each episode
    scores_window = deque(maxlen=80)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()[brain_name].vector_observations[0]
        score = 0
        
        
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]
            next_state, reward, done= env_info.vector_observations[0], env_info.rewards[0], env_info.local_done[0]
            agent.step(state, action, reward, next_state, done, BETA)
            agent.prioritize(state, action, reward, next_state, done)
#             state_list.append(state), action_list.append(action), reward_list.append(reward), next_state_list.append(next_state), done_list.append(done)            
            
#             if len(state_list)==BATCH_SIZE:
#                 agent.prioritize(state_list, action_list, reward_list, next_state_list, done_list)
#                 state_list, action_list, reward_list, next_state_list, done_list = [], [], [], [], []
            
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        BETA += min(BETA*0.001,1)
        #if BETA >0.4: print('alah akbar')
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 10 == 0:
            print('\nBeta =',BETA)
        
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            #print(BETA)
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), checkpoint_file)
            break
    np.savetxt(scores_file, np.array(scores))
    return scores

scores_list = []
for i in range(1,4):
    scores_list.append(dqn(checkpoint_file ='./model_files/per_checkpoint_'+str(i)+'.pth', scores_file = './scores_files/per_scores_'+str(i)+'.csv'))

Episode 10	Average Score: 0.50
Beta = 0.5050225601051261
Episode 20	Average Score: 0.20
Beta = 0.5100955724302714
Episode 30	Average Score: 0.23
Beta = 0.5152195437740509
Episode 40	Average Score: 0.33
Beta = 0.5203949860259325
Episode 50	Average Score: 0.26
Beta = 0.5256224162173756
Episode 60	Average Score: 0.30
Beta = 0.5309023565734823
Episode 70	Average Score: 0.36
Beta = 0.536235334565169
Episode 80	Average Score: 0.41
Beta = 0.5416218829618608
Episode 90	Average Score: 0.47
Beta = 0.5470625398847158
Episode 100	Average Score: 0.86
Beta = 0.5525578488603836
Episode 100	Average Score: 0.86
Episode 110	Average Score: 1.01
Beta = 0.5581083588753046
Episode 120	Average Score: 1.29
Beta = 0.5637146244305535
Episode 130	Average Score: 1.59
Beta = 0.5693772055972358
Episode 140	Average Score: 1.91
Beta = 0.5750966680724375
Episode 150	Average Score: 2.29
Beta = 0.5808735832357406
Episode 160	Average Score: 2.54
Beta = 0.5867085282063036
Episode 170	Average Score: 2.96
Beta = 0.592602085

Episode 300	Average Score: 7.70
Beta = 0.6748282394022579
Episode 300	Average Score: 7.70
Episode 310	Average Score: 7.96
Beta = 0.6816069701883264
Episode 320	Average Score: 8.36
Beta = 0.688453794140014
Episode 330	Average Score: 8.55
Beta = 0.6953693952613547
Episode 340	Average Score: 9.06
Beta = 0.7023544644272854
Episode 350	Average Score: 9.15
Beta = 0.7094096994526647
Episode 360	Average Score: 9.29
Beta = 0.7165358051619856
Episode 370	Average Score: 9.44
Beta = 0.7237334934597877
Episode 380	Average Score: 9.40
Beta = 0.731003483401777
Episode 390	Average Score: 9.51
Beta = 0.7383465012666608
Episode 400	Average Score: 9.41
Beta = 0.7457632806287038
Episode 400	Average Score: 9.41
Episode 410	Average Score: 9.59
Beta = 0.7532545624310112
Episode 420	Average Score: 9.78
Beta = 0.7608210950595516
Episode 430	Average Score: 10.40
Beta = 0.7684636344179206
Episode 440	Average Score: 10.78
Beta = 0.776182944002856
Episode 450	Average Score: 11.11
Beta = 0.7839797949805122
Episode 

In [ ]:
#env.reset()[brain_name].vector_observations[0]
import torch
from dqn_agent import Agent
from collections import deque
from time import sleep

agent = Agent(state_size=37, action_size=4, seed=0)

agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(5):
    state = env.reset()[brain_name].vector_observations[0]
    

    def get_priority_experience_batch(self):
        p_sum = np.sum(self.priority)
        prob = self.priority / p_sum
        sample_indices = random.choices(range(len(prob)), k=self.batch_size, weights=prob)for j in range(200):
        action = agent.act(state)
#        env.render()
        env_info = env.step(action)[brain_name]
        next_state, reward, done= env_info.vector_observations[0], env_info.rewards[0], env_info.local_done[0]
        
        sleep(0.04)
        
        
        if done:
            break 
            
env.close()

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```